In [8]:
import numpy as np
import pandas as pd
shanghai_ranking = pd.read_csv('data/Shanghai_ranking_data_2023.csv')
THE_ranking = pd.read_csv('data/THE_ranking_data_2023.csv')
qs_ranking = pd.read_csv('data/QS_ranking_data_2023.csv')

In [9]:
# THE_ranking = pd.read_csv('data/THE_ranking_data.csv')
# for index, item in enumerate(THE_ranking.RankTHE):
#     item = item.replace('–','-')
#     THE_ranking.at[index,'RankTHE'] = item
# THE_ranking.to_csv('output.csv')

In [10]:
THE_ranking

,Unnamed: 0,RankTHE,Name,No. of FTE students,No. of students per staff,International Students,Female:Male Ratio,Overall,Teaching,Research Environment,Research Quality,Industry,International Outlook
0,0,1,University of Oxford,"20,965",10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
1,1,2,Harvard University,"21,887",9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
2,2,=3,University of Cambridge,"20,185",11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,3,=3,Stanford University,"16,164",7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
4,4,5,Massachusetts Institute of Technology,"11,415",8.2,33%,40 : 60,94.2,90.7,93.6,99.8,90.9,89.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2142,2142,Reporter,York St John University,"6,315",18.6,12%,65 : 35,NaN,NaN,NaN,NaN,NaN,NaN
2143,2143,Reporter,"Yusuf Maitama Sule University, Kano","12,880",33.0,0%,48 : 52,NaN,NaN,NaN,NaN,NaN,NaN
2144,2144,Reporter,Zhytomyr Polytechnic State University,"3,869",15.4,1%,34 : 66,NaN,NaN,NaN,NaN,NaN,NaN
2145,2145,Reporter,Ziauddin University,"4,906",8.8,1%,63 : 37,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def merge(Rank1, Rank2, bias):
    import Levenshtein
    distance = np.empty((len(Rank1.Name),len(Rank2.Name)))
    Rank1 = Rank1.rename(columns={'Name':'Name1'})
    Rank2 = Rank2.rename(columns={'Name':'Name2'})
    for index1, rank1 in enumerate(Rank1.Name1):
        for index2, rank2 in enumerate(Rank2.Name2):
            distance[index1, index2] = Levenshtein.distance(rank1, rank2)
    merged_rank = pd.DataFrame({})
    for i in range(distance.shape[0]):
        j = np.argmin(distance[i, :])
        if distance[i, j] == bias:
            # print('i:',i,'j:',j)
            merged_row = pd.concat([Rank1.iloc[i], Rank2.iloc[j]], axis=0)
            merged_row = merged_row.to_frame().T
            merged_rank = pd.concat([merged_rank, merged_row])
    merged_rank = merged_rank.reset_index(drop=True)
    return merged_rank

def clean(merged_rank, row=[]):
    for i in row:
        merged_rank = merged_rank.drop(i)
    merged_rank = merged_rank.drop('Name2',axis=1)
    merged_rank = merged_rank.rename(columns={'Name1':'Name'}).reset_index(drop=True)
    return merged_rank

In [12]:
merged_rank1 = merge(shanghai_ranking,qs_ranking,0)
merged_rank1 = clean(merged_rank1)
merged_rank2 = merge(shanghai_ranking,qs_ranking,1)
merged_rank2 = clean(merged_rank2)
merged_rank = pd.concat([merged_rank1,merged_rank2])
merged_rank = merged_rank.reset_index(drop=True)
merged_rank

,RankShangHai,Name,Region,Regional rank,Total score,Alumni,Award,Hici,N&S,PUB,...,cpf rank,ifr score,ifr rank,isr score,isr rank,irn score,irn rank,ger score,ger rank,score scaled
0,1,Harvard University,us,1,100.0,100.0,92.6,100.0,100.0,100.0,...,2,76.9,228,66.9,212,100.0,1,100.0,1,97.6
1,2,Stanford University,us,2,74.8,45.6,88.1,73.6,82.8,67.5,...,9,99.8,74,60.3,235,96.3,55,100.0,2,98.5
2,4,University of Cambridge,gb,1,67.9,78.9,90.8,45.1,61.0,63.6,...,55,100.0,60,96.3,70,99.5,6,100.0,9,98.8
3,6,Princeton University,us,5,60.1,61.7,100.0,32.9,50.4,39.4,...,3,26.0,515,62.2,226,78.5,340,100.0,8,89.2
4,7,University of Oxford,gb,2,59.5,49.9,59.7,51.4,57.6,71.4,...,64,98.8,101,98.4,54,99.9,3,100.0,7,98.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,901-1000,Nicolaus Copernicus University,pl,5月9日,NaN,10.1,0.0,0.0,5.2,23.0,...,601+,3.6,601+,3.9,601+,71.5,441,5.2,601+,-
485,901-1000,Southern Cross University,au,31-34,NaN,21.9,0.0,0.0,7.3,15.1,...,393,73.5,243,88.1,120,57.3,601+,7.1,601+,-
486,901-1000,Tokyo University of Agriculture and Technology,jp,29-32,NaN,15.8,0.0,0.0,6.9,17.3,...,335,5.6,601+,6.4,601+,46.2,601+,5.1,601+,-
487,901-1000,Universidad Andr茅s Bello,cl,3月4日,NaN,8.6,0.0,6.6,2.9,18.7,...,601+,8.0,601+,2.2,601+,45.7,601+,4.3,601+,-


In [13]:
merged_rank1 = merge(merged_rank,THE_ranking,0)
merged_rank1 = clean(merged_rank1)
merged_rank2 = merge(merged_rank,THE_ranking,1)
merged_rank2 = clean(merged_rank2)
merged_rank = pd.concat([merged_rank1,merged_rank2])
merged_rank = merged_rank.reset_index(drop=True)
merged_rank

,RankShangHai,Name,Region,Regional rank,Total score,Alumni,Award,Hici,N&S,PUB,...,No. of FTE students,No. of students per staff,International Students,Female:Male Ratio,Overall,Teaching,Research Environment,Research Quality,Industry,International Outlook
0,1,Harvard University,us,1,100.0,100.0,92.6,100.0,100.0,100.0,...,"21,887",9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
1,2,Stanford University,us,2,74.8,45.6,88.1,73.6,82.8,67.5,...,"16,164",7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
2,4,University of Cambridge,gb,1,67.9,78.9,90.8,45.1,61.0,63.6,...,"20,185",11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,6,Princeton University,us,5,60.1,61.7,100.0,32.9,50.4,39.4,...,"8,279",8.0,23%,46 : 54,92.4,87.6,95.9,99.1,66.0,80.3
4,7,University of Oxford,gb,2,59.5,49.9,59.7,51.4,57.6,71.4,...,"20,965",10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,151-200,Aix Marseille University,fr,6月8日,NaN,19.0,0.0,18.6,21.5,48.1,...,"70,054",17.4,13%,60 : 40,47.0–48.7,40.5,24.9,72.3,41.5,63.2
406,301-400,Linkoping University,se,7月8日,NaN,19.1,0.0,13.2,10.6,32.0,...,"15,026",14.4,15%,52 : 48,48.9–51.1,23.5,39.8,78.0,61.8,70.8
407,301-400,University of Milano-Bicocca,it,9月10日,NaN,23.1,0.0,11.4,7.2,34.2,...,"29,057",29.2,5%,62 : 38,47.0–48.7,24.9,25.9,93.6,40.8,43.0
408,601-700,Palack媒 University Olomouc,cz,3月4日,NaN,20.5,0.0,6.6,6.6,21.8,...,"18,673",14.5,13%,69 : 31,24.4–29.7,19.6,19.9,40.0,37.8,59.3


In [14]:
type(merged_rank)

pandas.core.frame.DataFrame

In [15]:
sample_rank = merged_rank.loc[:,['Name','RankShangHai','RankQS','RankTHE']]
sample_rank

,Name,RankShangHai,RankQS,RankTHE
0,Harvard University,1,5,2
1,Stanford University,2,3,=3
2,University of Cambridge,4,2,=3
3,Princeton University,6,16=,7
4,University of Oxford,7,4,1
...,...,...,...,...
405,Aix Marseille University,151-200,490=,301-350
406,Linkoping University,301-400,320=,251-300
407,University of Milano-Bicocca,301-400,601-650,301-350
408,Palack媒 University Olomouc,601-700,651-700,1001-1200


In [16]:
sample_rank = merged_rank.loc[:,['Name','RankShangHai','RankQS','RankTHE']]

def level(rank,rankname):
    for index, item in enumerate(rank[rankname]):
        item=str(item)
        if (item == 'Reporter'):
            rank = rank.drop(index)
        if '=' in item:
            rank.at[index, rankname] = item.replace('=','')
        if '+' in item:
            rank.at[index, rankname] = item.replace('+','')
        if '-' in item:
            numbers = item.split('-')
            num1, num2 = map(int, numbers)
            # print(numbers)
            avg = str(int((num1+num2)/2))
            rank.at[index, rankname] = avg
    rank = rank.reset_index(drop=True)
    return rank
sample_rank = level(sample_rank,'RankQS')
sample_rank = level(sample_rank,'RankTHE')
sample_rank = level(sample_rank,'RankShangHai')

In [17]:
sample_rank

,Name,RankShangHai,RankQS,RankTHE
0,Harvard University,1,5,2
1,Stanford University,2,3,3
2,University of Cambridge,4,2,3
3,Princeton University,6,16,7
4,University of Oxford,7,4,1
...,...,...,...,...
404,Aix Marseille University,175,490,325
405,Linkoping University,350,320,275
406,University of Milano-Bicocca,350,625,325
407,Palack媒 University Olomouc,650,675,1100


In [24]:
medians = sample_rank.iloc[:,1:].median(axis=1)
sample_rank['Median'] = medians
sample_rank['Newrank'] = sample_rank.Median.rank(method='min',ascending=True)
sample_rank

,Name,RankShangHai,RankQS,RankTHE,Median,Newrank
0,Harvard University,1,5,2,2.0,1.0
1,Stanford University,2,3,3,3.0,2.0
2,University of Cambridge,4,2,3,3.0,2.0
3,Princeton University,6,16,7,7.0,5.0
4,University of Oxford,7,4,1,4.0,4.0
...,...,...,...,...,...,...
404,Aix Marseille University,175,490,325,325.0,163.0
405,Linkoping University,350,320,275,320.0,161.0
406,University of Milano-Bicocca,350,625,325,350.0,176.0
407,Palack媒 University Olomouc,650,675,1100,675.0,301.0


In [21]:
sample_rank2 = merged_rank.loc[:, ['Name','Alumni','Hici','PUB']]
sample_rank2

,Name,Alumni,Hici,PUB
0,Harvard University,100.0,100.0,100.0
1,Stanford University,45.6,73.6,67.5
2,University of Cambridge,78.9,45.1,63.6
3,Princeton University,61.7,32.9,39.4
4,University of Oxford,49.9,51.4,71.4
...,...,...,...,...
405,Aix Marseille University,19.0,18.6,48.1
406,Linkoping University,19.1,13.2,32.0
407,University of Milano-Bicocca,23.1,11.4,34.2
408,Palack媒 University Olomouc,20.5,6.6,21.8


In [25]:
sample_rank2 = merged_rank.loc[:, ['Name','Alumni','Hici','PUB']]
sample_rank2['RankAlumni'] = sample_rank2.Alumni.rank(method='average',ascending=False).astype(int)
sample_rank2 = sample_rank2.drop('Alumni',axis=1) 
sample_rank2['RankHici'] = sample_rank2.Hici.rank(method='average',ascending=False).astype(int)
sample_rank2 = sample_rank2.drop('Hici',axis=1) 
sample_rank2['RankPUB'] = sample_rank2.PUB.rank(method='average',ascending=False).astype(int)
sample_rank2 = sample_rank2.drop('PUB',axis=1) 
sample_rank2

,Name,RankAlumni,RankHici,RankPUB
0,Harvard University,1,1,1
1,Stanford University,7,2,16
2,University of Cambridge,2,10,23
3,Princeton University,3,30,126
4,University of Oxford,6,5,12
...,...,...,...,...
405,Aix Marseille University,186,97,71
406,Linkoping University,185,159,205
407,University of Milano-Bicocca,97,184,175
408,Palack媒 University Olomouc,154,288,356


In [26]:
medians = sample_rank2.iloc[:,1:].median(axis=1)
sample_rank2['Median'] = medians
sample_rank2['Newrank'] = sample_rank2.Median.rank(method='min',ascending=True)
sample_rank2

,Name,RankAlumni,RankHici,RankPUB,Median,Newrank
0,Harvard University,1,1,1,1.0,1.0
1,Stanford University,7,2,16,7.0,3.0
2,University of Cambridge,2,10,23,10.0,7.0
3,Princeton University,3,30,126,30.0,21.0
4,University of Oxford,6,5,12,6.0,2.0
...,...,...,...,...,...,...
405,Aix Marseille University,186,97,71,97.0,91.0
406,Linkoping University,185,159,205,185.0,186.0
407,University of Milano-Bicocca,97,184,175,175.0,171.0
408,Palack媒 University Olomouc,154,288,356,288.0,278.0
